# Применение TensorRT для оптимизации нейросетей
Изначально TensorRT существовала в видел отдельной библиотеки, но относительно недавно её функционал был в том числе
интегрирован в Tensorflow. При такой интеграции на выходе мы всё еще получаем
Tensorflow модель, просто части её графа будут оптимизированы и вычислены (во время инференса) с помощью TensorRT. 

In [1]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.13.0


# Импорт TensorRT

In [2]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

# Создание модели
Создадим свёрточную нейронную сеть с большим количеством свёрточных слоёв.
Кроме того, TensorRT оптимизации необходимо фиксировать размер входа. Делаем это с помощью метода _set_inputs()

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model._set_inputs(np.zeros((1, 28, 28, 1), dtype=np.float32))

# Сохранение модели

In [4]:
model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


# Оптимизация модели с помощью TensorFlow
Сначаласоздадим TensorRT конветтер (converter) и укажем ему путь до нашей неоптимизированной модели. Если нужно как-то еще 
сконфигурировать процесс оптимизации, можно передать дополнительные параметры в конструктор TrtGraphConverterV2, но мы
будем использовать параметры по умолчанию.
После этого просто вызываем метод convert() и TensorRT применит все свои стратегии для оптимизации нашей модели. Важно
запускать этот на GPU, так как TensorRT работает только с GPU.

In [5]:
converter = trt.TrtGraphConverterV2(input_saved_model_dir='saved_model')
converter.convert()
converter.save('saved_model_trt')

ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


RuntimeError: Tensorflow has not been built with TensorRT support.

In [6]:
model_trt = tf.keras.models.load_model('saved_model_trt')

# Warm-up
_ = model_trt(np.zeros((1, 28, 28, 1), dtype=np.float32))

OSError: No file or directory found at saved_model_trt

# Сравнение скорости работы двух моделей

In [7]:
%%timeit -n 10 -r 10
q = model(np.zeros((1, 28, 28, 1), dtype=np.float32))

The slowest run took 6.30 times longer than the fastest. This could mean that an intermediate result is being cached.
8.14 ms ± 8.38 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [8]:
%%timeit -n 10 -r 10
q = model_trt(np.zeros((1, 28, 28, 1), dtype=np.float32))

NameError: name 'model_trt' is not defined